In [1]:
# coding=utf-8
import random
import time

from scrapy.http import TextResponse
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

启动Chrome浏览器实例：
1）进入Chrome浏览器目录下，运行cmd

2）运行Chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\selenum\AutomationProfile"

In [76]:
# 配置Chrome浏览器选项
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')  # 远程调控模式启用
# options.add_argument('--incognito')  # 隐身/无痕模式启用
browser = webdriver.Chrome(options=options)
action = ActionChains(browser)

In [77]:
# 定义全局变量
key_word = ""
num = 0

In [78]:
def selenium_test():
    global key_word, num  # 使用全局变量
    browser.get('https://www.xiaohongshu.com/explore')

    print("即将开始检查小红书登录状态...")
    # 定义循环：检查是否成功登录小红书
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            time.sleep(10)
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            continue
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    key_word = input("搜索话题关键词：")
    num = input("笔记爬取数量：")
    num = int(num)

    url = f'https://www.xiaohongshu.com/search_result?keyword={key_word}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)

    # 定义循环：检查页面是否加载成功
    while True:
        if key_word in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(2)

    # 通过模拟点击更改排序方式
    sort_order = ['综合', '最新', '最热']
    print("请在文本框中根据提示输入对应数字来选择排序方式。")
    for i, order in enumerate(sort_order, 1):
        print(f'{i}.{order}')
    selected_order = input("请选择排序方式:")

    try:
        selected_order_index = int(selected_order)
        selected_order_text = sort_order[selected_order_index - 1]
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()  # 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()  # 模拟鼠标点击
        print('已选择排序方式为:', selected_order_text)
        print('检查时间:', time.ctime())
    except Exception as e:
        print(e)


if __name__ == '__main__':
    selenium_test()

即将开始检查小红书登录状态...
暂未登录，请手动登录
检查时间: Sat Apr 27 22:39:47 2024
暂未登录，请手动登录
检查时间: Sat Apr 27 22:39:57 2024
暂未登录，请手动登录
检查时间: Sat Apr 27 22:40:07 2024
登录成功
检查时间: Sat Apr 27 22:40:07 2024
即将开始检查网页加载状态...
如果网页进入人机验证页面，请先手动完成验证。
请在文本框中根据提示输入搜索关键词和笔记爬取数量。
加载成功
检查时间: Sat Apr 27 22:40:32 2024
请在文本框中根据提示输入对应数字来选择排序方式。
1.综合
2.最新
3.最热
已选择排序方式为: 最热
检查时间: Sat Apr 27 22:40:37 2024


In [79]:
URL_list = []
userURL_list = []

wait_time = random.uniform(5, 8)  # 生成随机等待时间


# 定义函数：解析网页内容，爬取数据
def parsePage(key_word):
    response = TextResponse(url=browser.current_url, body=key_word, encoding='utf-8')  # 将key_word转换为Response对象
    selector = Selector(response)
    divs = selector.xpath('//div[contains(@class, "feeds-container")]/section/div')  # 选中网页中包含笔记信息的部分
    
    print(f'当前页面包含{len(divs)}条笔记')

    # 定义循环：遍历divs获取每一篇笔记的信息
    for div in divs:
        # 选择并提取网页数据
        try:
            url = div.xpath('./div/a/@href').extract_first().split('/')[-1]  # 笔记URL
            user_url = div.xpath('./div/div/a/@href').extract_first()  # 用户URL
            
            print(url)

            URL_list.append(url)
            userURL_list.append(user_url)

            time.sleep(0.35)
            
        except:
            pass
        
    print(len(set(URL_list)))


# 定义循环：检查是否已经爬取足够数量的笔记，或是否已经达到页面底部
while len(URL_list) < num:
    if '- THE END -' in browser.page_source:
        print(f"当前与{key_word}有关的笔记数量少于 {num}")
        print('检查时间:', time.ctime())
        break

    parsePage(browser.page_source)  # 解析当前页面
    time.sleep(5)
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')  # 模拟鼠标滚动
    time.sleep(wait_time)

if len(URL_list) > num:
    URL_list = URL_list[:num]
    userURL_list = userURL_list[:num]

当前页面包含18条笔记
65fe7252000000000d00fd8d
65eee368000000000d00f228
65ff79b70000000014007541
65432d74000000002202da48
65f942d200000000120316a1
661f8053000000000d031582
65bb8c01000000002c016da0
65f71948000000001203c8bf
65f15bfd0000000014006f7c
6571c1d10000000006028352
65db0ec40000000007026d1b
65c0a4dd000000000a0102f7
660fe456000000001a00d40b
660a7716000000001b00f515
65d7ff0c000000000102ae4d
6592d906000000001102d414
6517f53b00000000220286cf
17
当前页面包含18条笔记
660fe456000000001a00d40b
660a7716000000001b00f515
65d7ff0c000000000102ae4d
6592d906000000001102d414
6517f53b00000000220286cf
660a49e5000000001b00a258
659bc5670000000011030fad
66053b00000000001203c0de
66115913000000001a013d2b
64be41b3000000001201a78f
65bb8e11000000002c037665
65ff7e000000000012020c60
65b9f52c000000002c03c8fc
6595425c0000000011017d3b
654df10d0000000032005b0c
65d4beed0000000007025c71
66000446000000000d00faa0
29
当前页面包含18条笔记
65ff7e000000000012020c60
65b9f52c000000002c03c8fc
6595425c0000000011017d3b
654df10d0000000032005b0c
65d4beed

In [80]:
print(len(set(URL_list)),len(URL_list))

201 289


In [81]:
userURL_list_cleaned = [url.split('/')[-1] for url in userURL_list]

ID_FansNr_dic = {}

wait_time = random.uniform(1, 2)

# 定义循环：遍历items中的每一篇笔记
for user_url in set(userURL_list_cleaned):
    whole_url = 'https://www.xiaohongshu.com/user/profile/' + user_url  # 构造完整个人页面URL
    browser.get(whole_url)

    WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="userPostedFeeds"]//section')))  # 等待笔记页面加载完成
    # 解析完成加载的网页内容
    html = browser.page_source
    selector = Selector(text=html)

    # 选择并提取网页数据
    author_fans_nr = selector.xpath(
            '//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()  # 作者粉丝数量
    author_ID = selector.xpath(
            '//*[@class="basic-info"]/div[@class="user-basic"]/div[@class="user-content"]/span[@class="user-redId"]/text()').extract_first()  # 作者小红书号
    
    author_ID = author_ID.split('：')[-1]
    
    print('小红书号:',author_ID, '粉丝数:',author_fans_nr)
    ID_FansNr_dic[author_ID] = author_fans_nr

    time.sleep(wait_time)

小红书号: 94920717172 粉丝数: 555
小红书号: 821646244 粉丝数: 21.2万
小红书号: 819084532 粉丝数: 5.6万
小红书号: 495623653 粉丝数: 14.8万
小红书号: 626912451 粉丝数: 2.2万
小红书号: liliuli225 粉丝数: 2.4万
小红书号: 855114773 粉丝数: 1.1万
小红书号: 111253447 粉丝数: 520
小红书号: 603517312 粉丝数: 10W+
小红书号: 8991686587 粉丝数: 2807
小红书号: 973923103 粉丝数: 6.9万
小红书号: Hiphop520jhh 粉丝数: 6149
小红书号: 3503400258 粉丝数: 2万
小红书号: 5338270239 粉丝数: 3.2万
小红书号: 569311400 粉丝数: 1.1万
小红书号: gaizao2954 粉丝数: 2.1万
小红书号: 7874088451 粉丝数: 3782
小红书号: 9535196916 粉丝数: 8076
小红书号: 956904707 粉丝数: 100+
小红书号: 279745954 粉丝数: 1.4万
小红书号: 638748516 粉丝数: 489
小红书号: 8932695237 粉丝数: 1.4万
小红书号: 11807011832 粉丝数: 2104
小红书号: 993060214 粉丝数: 1.6万
小红书号: 3406182043 粉丝数: 2055
小红书号: 888885472 粉丝数: 1.1万
小红书号: 9643244992 粉丝数: 3.1万
小红书号: zhaopipi123 粉丝数: 3.9万
小红书号: 4141551466 粉丝数: 717
小红书号: 9749603654 粉丝数: 4537
小红书号: 394287298 粉丝数: 158
小红书号: 376450816 粉丝数: 628
小红书号: XANYPSHOP 粉丝数: 2704
小红书号: 539606069 粉丝数: 1W+
小红书号: nnan420 粉丝数: 32万
小红书号: 433220106 粉丝数: 5372
小红书号: CBJ_Messi 粉丝数: 184
小红书号: 5340546834 粉丝数: 3.5万
小

In [86]:
print(len(ID_FansNr_dic))

182


In [87]:
for author_ID, author_fans_nr in ID_FansNr_dic.items():
    author_fans_nr = str(author_fans_nr)
    if author_fans_nr[-1] == '+':
        ID_FansNr_dic[author_ID] = author_fans_nr[:-1]
    if author_fans_nr[-1] == '万' or author_fans_nr[-1] == 'W':
        ID_FansNr_dic[author_ID] = float(author_fans_nr[:-1]) * 10000
    if author_fans_nr[-1] == 'K':
        ID_FansNr_dic[author_ID] = float(author_fans_nr[:-1]) * 1000

In [88]:
# 写入Excel
author_ID_list = []
author_fans_nr_list = []

for author_ID, author_fans_nr in ID_FansNr_dic.items():
    if int(author_fans_nr)>1000:
        author_ID_list.append(author_ID)
        author_fans_nr_list.append(author_fans_nr)

data = {
    "小红书ID": author_ID_list,
    "粉丝数量": author_fans_nr_list,
}
content = pd.DataFrame(data)
content.to_excel('data.xlsx', index=False)